In [ ]:
import os
from os.path import exists
import cv2
from PIL import Image
import numpy as np
import zipfile
from sklearn.metrics import confusion_matrix
import seaborn
from sklearn.metrics import accuracy_score

SRC_PATH = os.getcwd()
ASSETS_PATH = os.path.join(SRC_PATH, 'Assets')

Image pre-processing

In [ ]:
# Extracting the zip file

yale_faces_zip_path = os.path.join(ASSETS_PATH, "yalefaces.zip")
zip = zipfile.ZipFile(file=yale_faces_zip_path, mode = 'r')
zip.extractall(os.path.join(ASSETS_PATH, "Data"))
zip.close()


In [ ]:
def pre_process_images() -> None:
    """Pre-process images from a folder to be used in the LBPH classifier.

    Args:
        images_folder_path (str): Path to the folder containing the images.
    """
    image_paths = [os.path.join(ASSETS_PATH, "Data", "yalefaces", "train", img_path) for img_path in os.listdir(os.path.join(ASSETS_PATH, "Data", "yalefaces", "train"))]
    faces = []
    ids = []
    for path in image_paths:
        image = Image.open(path).convert('L')
        imagem_np = np.array(image, 'uint8')
        id = int(os.path.split(path)[1].split('.')[0].replace('subject', ''))
        ids.append(id)
        faces.append(imagem_np)

    return np.array(ids), faces

In [ ]:
ids, faces = pre_process_images()

In [ ]:
from cv2 import Mat

def show_image(image_path:str = None, cv2_image:Mat = None, title:str="Image") -> None:
    """Shows an image in a window with cv2. And waits for a key to be pressed to close the window.
    The path to the image or the image object must be provided.

    Args:
        image_path (str, optional): Path to the image to show. Defaults to None.
        cv2_image (Mat, optional): Image object to show. Defaults to None.
        title (str, optional): Title of the window containing the shown image. Defaults to "Image".
    """
    if image_path is None and cv2_image is None:
        raise Exception("No image to show")
    
    if image_path is not None:
        image_to_show = cv2.imread(image_path)
    else: 
        image_to_show = cv2_image
    
    cv2.imshow(title, image_to_show)
    cv2.waitKey(0)
    cv2.destroyWindow(title)

LBPH training and testing

In [ ]:
# train the classifier

if exists(os.path.join(ASSETS_PATH, "lbph_classifier.yml")):
    print("Loading classifier from file")
    lbph_classifier = cv2.face.LBPHFaceRecognizer_create()
    lbph_classifier.read(os.path.join(ASSETS_PATH, "lbph_classifier.yml"))
else:
    print("Training classifier")
    lbph_classifier = cv2.face.LBPHFaceRecognizer_create(radius=4, neighbors=14, grid_x=9, grid_y=9)
    lbph_classifier.train(faces, ids)
    lbph_classifier.write(os.path.join(ASSETS_PATH, "lbph_classifier.yml"))

In [ ]:
# test the classifier

test_image = os.path.join(ASSETS_PATH, "Data", "yalefaces", "test", "subject05.surprised.gif")

image = Image.open(test_image).convert('L')
np_image = np.array(image, 'uint8')

prediction = lbph_classifier.predict(np_image)

expected_output = int(os.path.split(test_image)[1].split('.')[0].replace('subject', ''))

cv2.putText(np_image, 'Pred: ' + str(prediction[0]), (10,30), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,255,0))
cv2.putText(np_image, 'Exp: ' + str(expected_output), (10,50), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,255,0))

show_image(cv2_image=np_image)

In [ ]:
def get_predictions_and_expected_outputs() -> tuple[np.array, np.array]:
    """Predicts the output of the LBPH classifier for the test images and gets the expected outputs.

    Returns:
        tuple[np.array, np.array]: Tuple containing the predictions and the expected outputs.
    """
    paths = [os.path.join(ASSETS_PATH, "Data", "yalefaces", "test", f) for f in os.listdir(os.path.join(ASSETS_PATH, "Data", "yalefaces", "test"))]
    predictions = []
    expected_outputs = []
    for path in paths:
        image = Image.open(path).convert('L')
        np_image = np.array(image, 'uint8')
        prediction, _ = lbph_classifier.predict(np_image)
        expected_output = int(os.path.split(path)[1].split('.')[0].replace('subject', ''))
        
        predictions.append(prediction)
        expected_outputs.append(expected_output)
    print(predictions) 
    print(expected_outputs)
    return np.array(predictions), np.array(expected_outputs)

In [ ]:
# Evaluate the classifier

predictions, expected_outputs = get_predictions_and_expected_outputs()
accuracy_score(predictions, expected_outputs)
cm = confusion_matrix(predictions, expected_outputs)
seaborn.heatmap(cm, annot=True);